# Lecture 2: Manipulating and combining data

_Please sign attendance sheet_

## Feeling overwhlemed?

Reminder that learning to code is like learning a spoken language. It's not obvious, and people will pick it up at different speeds at different spots. Try:

- Taking notes in the lecture notebooks
- Using [another Python/pandas learning resource](https://python-public-policy.afeld.me/en/{{school_slug}}/resources.html)
   - Hear things explained another way
   - Ask in [Ed Discussions]({{discussions_url}}) if others have recommendations
- [Comment-driven development](https://www.sitepoint.com/comment-driven-development/)
   - Otherwise, trying to do two steps in your head:
      1. Figuring out the logic
      1. Figuring out the syntax

Small example of comment-driven development:

```python
# find valid ZIP codes
# filter the DataFrame to only invalid ZIP codes
```

## Data cleaning

> Data Cleansing is a process of removing or fixing incorrect, malformed, incomplete, duplicate, or corrupted data

https://hevodata.com/learn/data-cleansing-a-simplified-guide/

When have you needed to clean data?

What are continuous values?

What are categorical values?

### Things to check for

From [my workshop on data cleaning](https://github.com/afeld/data-cleaning):

- Missing data
   - Empty values
- Bad (junk) values
   - Duplicates
   - Mismatched types/formatting
- Categorical values
   - Uniqueness (cardinality)
   - Value counts
- Continuous values
   - Ranges
   - Spread (distribution)

Notes:

- "Values" in this case can be a single cell (in the spreadsheet sense) or a whole row
- "Missing" or "duplicates" can be columns (Series), tables (DataFrames), rows, or cells

### Data cleaning [mnemonic](https://literaryterms.net/mnemonic/)

- Empty
- Bad
- Unique
- Spread

## Setup

In [1]:
import pandas as pd

In [2]:
# Display more rows and columns in the DataFrames
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

### Read our cleaned 311 Service Requests dataset

In [3]:
url = "https://storage.googleapis.com/python-public-policy2/data/311_requests_2018-19_sample_clean.csv.zip"
requests = pd.read_csv(url)

/var/folders/kr/nx0m1j811kz5vy8c87ffchzr0000gn/T/ipykernel_3268/610958175.py:2: DtypeWarning: Columns (8,20,31,34) have mixed types. Specify dtype option on import or set low_memory=False.
  requests = pd.read_csv(url)


## Dealing with dtypes

More data cleaning!

![Minion character vacuuming](https://impulsecreative.com/hs-fs/hubfs/cleaning-minion-gif.gif?width=490&name=cleaning-minion-gif.gif)

```
DtypeWarning: Columns (8,20,31,34) have mixed types.
```

In [4]:
requests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499958 entries, 0 to 499957
Data columns (total 41 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unique Key                      499958 non-null  int64  
 1   Created Date                    499958 non-null  object 
 2   Closed Date                     476140 non-null  object 
 3   Agency                          499958 non-null  object 
 4   Agency Name                     499958 non-null  object 
 5   Complaint Type                  499958 non-null  object 
 6   Descriptor                      492496 non-null  object 
 7   Location Type                   392573 non-null  object 
 8   Incident Zip                    480394 non-null  object 
 9   Incident Address                434529 non-null  object 
 10  Street Name                     434504 non-null  object 
 11  Cross Street 1                  300825 non-null  object 
 12  Cross Street 2  

In [5]:
list(requests["Incident Zip"].unique())

['11235',
 '11221',
 '11693',
 '11216',
 '10465',
 '11367',
 '10459',
 '11101',
 '11362',
 '10014',
 '11234',
 '11436',
 '10305',
 '10467',
 '11208',
 '10451',
 '11419',
 '11237',
 '11220',
 '10469',
 '11385',
 '10470',
 '11694',
 '10036',
 nan,
 '10473',
 '11435',
 '10040',
 '10472',
 '11225',
 '10019',
 '11434',
 '11226',
 '10010',
 '11211',
 '11421',
 '10026',
 '10013',
 '11423',
 '10002',
 '10453',
 '11213',
 '11104',
 '11249',
 '11361',
 '11233',
 '11224',
 '11374',
 '10025',
 '10022',
 '11214',
 '11209',
 '11366',
 '10304',
 '10027',
 '11378',
 '11206',
 '10021',
 '11364',
 '10065',
 '10456',
 '10314',
 '10312',
 '11212',
 '11379',
 '10462',
 '11231',
 '10460',
 '11416',
 '10001',
 '11357',
 '11413',
 '11210',
 '11217',
 '11223',
 '11417',
 '11418',
 '11218',
 '11230',
 '11207',
 '11691',
 '10468',
 '10007',
 '10310',
 '10306',
 '11103',
 '11105',
 '11433',
 '11203',
 '10307',
 '11229',
 '11372',
 '10032',
 '11420',
 '10017',
 '10301',
 '11368',
 '11201',
 '11365',
 '11422',
 '10

ZIP codes _look_ numeric, but aren't really.

[Read the ZIP codes in as strings.](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#text-data-types)

In [6]:
requests2 = pd.read_csv(url, dtype={"Incident Zip": "string"})

/var/folders/kr/nx0m1j811kz5vy8c87ffchzr0000gn/T/ipykernel_3268/1354368569.py:1: DtypeWarning: Columns (20,31,34) have mixed types. Specify dtype option on import or set low_memory=False.
  requests2 = pd.read_csv(url, dtype={"Incident Zip": "string"})


We fixed the dtype warning for column 8 (`Incident Zip`).

In [7]:
list(requests2["Incident Zip"].unique())

['11235',
 '11221',
 '11693',
 '11216',
 '10465',
 '11367',
 '10459',
 '11101',
 '11362',
 '10014',
 '11234',
 '11436',
 '10305',
 '10467',
 '11208',
 '10451',
 '11419',
 '11237',
 '11220',
 '10469',
 '11385',
 '10470',
 '11694',
 '10036',
 <NA>,
 '10473',
 '11435',
 '10040',
 '10472',
 '11225',
 '10019',
 '11434',
 '11226',
 '10010',
 '11211',
 '11421',
 '10026',
 '10013',
 '11423',
 '10002',
 '10453',
 '11213',
 '11104',
 '11249',
 '11361',
 '11233',
 '11224',
 '11374',
 '10025',
 '10022',
 '11214',
 '11209',
 '11366',
 '10304',
 '10027',
 '11378',
 '11206',
 '10021',
 '11364',
 '10065',
 '10456',
 '10314',
 '10312',
 '11212',
 '11379',
 '10462',
 '11231',
 '10460',
 '11416',
 '10001',
 '11357',
 '11413',
 '11210',
 '11217',
 '11223',
 '11417',
 '11418',
 '11218',
 '11230',
 '11207',
 '11691',
 '10468',
 '10007',
 '10310',
 '10306',
 '11103',
 '11105',
 '11433',
 '11203',
 '10307',
 '11229',
 '11372',
 '10032',
 '11420',
 '10017',
 '10301',
 '11368',
 '11201',
 '11365',
 '11422',
 '1

### Find invalid ZIP codes

Use a [regular expression (regex)](https://regexone.com/) to [find strings that match a pattern](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#testing-for-strings-that-match-or-contain-a-pattern):

```
^\d{5}(?:-\d{4})?$
│ │ │  │        │└─ end of string
│ │ │  │        └─ optional
│ │ │  └─ capture group
│ │ └─ count
│ └─ numeric/digit character
└─ start of string
```

[regex101](https://regex101.com/) is useful for testing them.

In [8]:
# find valid ZIP codes
valid_zips = requests2["Incident Zip"].str.contains(r"^\d{5}(?:-\d{4})?$")

# filter the DataFrame to only invalid ZIP codes
invalid_zips = valid_zips == False
requests_with_invalid_zips = requests2[invalid_zips]
requests_with_invalid_zips["Incident Zip"]

55017     HARRISBURG
58100         N5X3A6
80798         100000
120304           IDK
123304          1801
173518     14614-195
192034        979113
201463           100
207158          8682
216745        000000
325071      NJ 07114
425985          1101
441166         DID N
Name: Incident Zip, dtype: string

[Clear](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html#inserting-missing-data) any invalid ZIP codes:

In [9]:
requests2.loc[invalid_zips, "Incident Zip"] = None

Additonal data cleaning tips:

- Hard part is finding what needs to be done
- Will be specific to your use case
- Document what you did, since it will affect your results

## [In-class exercise](https://python-public-policy.afeld.me/en/{{school_slug}}/lecture_2_exercise.html)

## [Concatenation](https://pandas.pydata.org/docs/user_guide/merging.html#concat)

In [10]:
revenue_q1 = pd.DataFrame(
    {
        "month": ["Jan", "Feb", "Mar"],
        "revenue": [175589, 189645, 163423],
    }
)
revenue_q1

,month,revenue
0,Jan,175589
1,Feb,189645
2,Mar,163423


In [11]:
revenue_q2 = pd.DataFrame(
    {
        "month": ["Apr", "May", "Jun"],
        "revenue": [14567, 15863, 17999],
    }
)
revenue_q2

,month,revenue
0,Apr,14567
1,May,15863
2,Jun,17999


In [12]:
pd.concat([revenue_q1, revenue_q2])

,month,revenue
0,Jan,175589
1,Feb,189645
2,Mar,163423
0,Apr,14567
1,May,15863
2,Jun,17999


## Simple [merge](https://pandas.pydata.org/docs/user_guide/merging.html#merge)

_I had [Copilot](https://code.visualstudio.com/docs/copilot/overview) generate the DataFrames, so no idea if the numbers are real._

In [13]:
populations = pd.DataFrame(
    {
        "Country": ["China", "India", "Pakistan"],
        "Population": [1444216107, 1393409038, 220892331],
    }
)

populations

,Country,Population
0,China,1444216107
1,India,1393409038
2,Pakistan,220892331


In [14]:
gdps = pd.DataFrame(
    {
        "Country": ["China", "India", "United States", "Indonesia", "Pakistan"],
        "GDP": [14342903, 2875142, 21433226, 1058393, 263687],
    }
)
gdps

,Country,GDP
0,China,14342903
1,India,2875142
2,United States,21433226
3,Indonesia,1058393
4,Pakistan,263687


To join dataframes together, we will use the [pandas `.merge()` function](https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/08_combine_dataframes.html#join-tables-using-a-common-identifier).

![merge diagram](https://pandas.pydata.org/pandas-docs/stable/_images/08_merge_left.svg)

`.merge()` is comparable to:

- [SQL `JOIN`](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html#join)
- [Spreadsheet `VLOOKUP`](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_spreadsheets.html#merging)

In general, called ["record linkage" or "entity resolution"](https://en.wikipedia.org/wiki/Record_linkage).

In [15]:
pd.merge(populations, gdps, on="Country")

,Country,Population,GDP
0,China,1444216107,14342903
1,India,1393409038,2875142
2,Pakistan,220892331,263687


In [16]:
pd.merge(populations, gdps, on="Country", how="outer")

,Country,Population,GDP
0,China,1.444216e+09,14342903
1,India,1.393409e+09,2875142
2,Indonesia,NaN,1058393
3,Pakistan,2.208923e+08,263687
4,United States,NaN,21433226


[Different types of merges](https://www.geeksforgeeks.org/different-types-of-joins-in-pandas/)

## In-class exercise 2

Let's compute the migrants as a percent of total population by country using [UN data](https://data.un.org/).

## [Homework 2](https://python-public-policy.afeld.me/en/{{school_slug}}/hw_2.html)